In [37]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import os

model = SentenceTransformer("intfloat/multilingual-e5-large")
dim = model.get_sentence_embedding_dimension()


In [39]:
import os
import json

chunks = []

folder = "chunks"
for filename in os.listdir(folder):
    if filename.endswith(".json"):
        path = os.path.join(folder, filename)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)  # could be a list of chunks
            if isinstance(data, list):
                chunks.extend(data)  # add all chunks
            elif isinstance(data, dict):
                chunks.append(data)  # add single chunk

print(f"✅ Loaded {len(chunks)} chunks from {folder}")

texts = [c["content"] for c in chunks]
metadata = [
    {
        "id": c["chunk_id"],
        "title": c["title"],
        "source": c["source"],
        "text": c["content"]
    }
    for c in chunks
]

✅ Loaded 38 chunks from chunks


In [40]:
from tqdm import tqdm

embeddings = model.encode(
    [f"passage: {t}" for t in tqdm(texts, desc="Encoding chunks")],
    batch_size=16,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")


Encoding chunks: 100%|██████████| 38/38 [00:00<?, ?it/s]


In [41]:
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

os.makedirs("storage", exist_ok=True)
faiss.write_index(index, "storage/index.faiss")

with open("storage/metadata.jsonl", "w", encoding="utf-8") as f:
    for m in metadata:
        f.write(json.dumps(m, ensure_ascii=False) + "\n")

print(f"✅ Stored {len(metadata)} chunks into FAISS index")

✅ Stored 38 chunks into FAISS index


In [42]:
index = faiss.read_index("storage/index.faiss")
with open("storage/metadata.jsonl", "r", encoding="utf-8") as f:
    metadata = [json.loads(line) for line in f]

query = "شو هي الأسماء اللي علمها الله سبحانه وتعالى للنبي آدم؟"
query = "مسقط رأس السيد هاشم"

q_emb = model.encode(
    [f"query: {query}"],
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

D, I = index.search(q_emb, k=3)  # top-3 results
for score, idx in zip(D[0], I[0]):
    print("Score:", score)
    print("Chunk:", metadata[idx]["text"])
    print("---")


Score: 0.80997866
Chunk: أنا السيد هاشم صفي الدين، وُلدت عام 1964 في دير قانون النهر بجنوب لبنان. نشأت في أسرة ملتزمة، ودرست العلوم الدينية في النجف وقم. أعتز بانتمائي للإسلام والمذهب الشيعي.
---
Score: 0.80045736
Chunk: بسم الله الرحمن الرحيم. والصلاة والسلام على سيدنا أبي القاسم محمد وعلى ال بيته الطيبين الطاهرين. ذكرنا لغاية الان مشهدين من مشاهد خلق ادم وعلم ادم والحجج التي القاها الله عز وجل على الملائكة، وكيف عاند ابليس وخرج عن طاعة الله عز وجل وأبى أن يسجد لادم سلام الله عليه متعللا بأنه من نار، وأن ادم عليه السلام من طين. بعد هذا أصبح ادم موجودا وستبدأ وظائفه وستبدأ المهام تنقى عليه. المهمة الاولى التي ألقيت على ادم هو ان يسكن في الجنة. الجنة التي ستكون سببا لهبوطه الى الارض في نهاية المطاف. الله عز وجل يريد من ادم ان يهبط الى الارض وهذا هو المقصد الاساسي من خلق ادم. من وجود الحياة الدنيا على هذه الارض وتكاثر البشرية. لكن هذا لا يتم تلقائيا. لابد ان يكون للانسان دور فيه لحتى تكون هذه الوظيفة منتزجة بين ما يطلبه الله عز وجل وبين ما يفعله الانسان.
---
Score: 0.78666365
Chunk: ولعل

In [24]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_answer(query, retrieved_chunks, model="gpt-4o-mini"):

    context = "\n\n".join([c["text"] for c in retrieved_chunks])

    messages = [
        {
            "role": "system",
            "content": (
                "أنت مساعد يجيب بناءا على فكر السيد هاشم صفي الدين."
                "اعتمد في اجابتك على النصوص المتوفرة في السياق. "
                "إذا لم يكن الجواب واضحا وكاملا في السياق، قل أنك لا تعرف. "
                "تكلم باحترام عن الشخصيات الشيعية, مع ذكر الألقاب المناسبة."
                "أجب دائمًا باللغة العربية الفصحى الواضحة."
            ),
        },
        {
            "role": "user",
            "content": f"السياق:\n{context}\n\nالسؤال: {query}",
        },
    ]

    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2,
    )

    return response.choices[0].message.content.strip()


In [25]:
query = "شو هي الأسماء اللي علمها الله سبحانه وتعالى للنبي آدم؟"
query = "ما رأي السيد هاشم بالامام علي؟"

q_emb = model.encode([
    f"query: {query}"
], convert_to_numpy=True, normalize_embeddings=True).astype("float32")

k = 3
D, I = index.search(q_emb, k)
results = [metadata[idx] for idx in I[0]]

retrieved_chunks = results
answer = generate_answer(query, retrieved_chunks)
print(answer)

TypeError: sequence item 0: expected str instance, dict found